In [ ]:
from repenseai.genai.agent import Agent
from repenseai.genai.tasks.api import Task
from repenseai.genai.tasks.workflow import Workflow
from repenseai.genai.tasks.conditional import BooleanConditionalTask
from repenseai.genai.tasks.function import FunctionTask

from pydantic import BaseModel


def check_children(context: dict):
    children = context["user_output"]["children"]
    return children > 0

def return_exit(context: dict):
    return "Exit"


class User(BaseModel):
    name: str
    age: int
    children: int


sample1 = """
My name is Samuel, I'm 36 years old and I dont have kids.
"""

sample2 = """
You are talking to Karen. She is a 29-year-old woman who has been a teacher for the past 12 years.
She has two children, who are 12 and 15 years old.
"""

agent = Agent(
    model="gpt-4o",
    model_type="chat",
    json_mode=True,
    json_schema=User,
)

sales_agent = Agent(
    model="claude-3-7-sonnet-20250219",
    model_type="chat",
)

task = Task(
    user="Extract the data from the text: {user}",
    agent=agent,
    simple_response=True
)

sales_task = Task(
    user="Create a sales pitch for {user_output} to travel to disneyland.",
    agent=sales_agent,
    simple_response=True
)

bool_task = BooleanConditionalTask(
    condition=check_children,
    true_task=sales_task,
    false_task=FunctionTask(return_exit),
)

workflow = Workflow(
    [
        [task, "user_output"],
        [bool_task, "sales_pitch"],
    ]
)
    
response = workflow.run({"user": sample1})

print()
print(response["sales_pitch"])